In [ ]:
import src.graph

In [ ]:
input = {
    'file_name': 'smaller_data.json',
    'user_id' : 'p007',
    'user_session_id' : '1',
    'data_info_from_user' : "json array of events triggered by all minecraft players in a server in choronological order",
    'messages' : []
}

In [ ]:
workflow_complied = src.graph.get_feild_name_correcting_task_graph()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(workflow_complied.get_graph().draw_mermaid_png()))
    #save it to a file
    workflow_complied.get_graph().draw_mermaid_png(output_file_path="workflow.png")
except Exception:
    pass

In [ ]:
out = workflow_complied.invoke(input)

In [ ]:
"""with open("temp/p007/1/out.json", "r") as f:
    data = f.read()
    print(len(data))"""

In [ ]:
"""with open("temp/p007/1/data.json", "r") as f:
    data = f.read()
    print(len(data))"""

In [1]:
from gcloud import storage
from oauth2client.service_account import ServiceAccountCredentials
import os, dotenv

dotenv.load_dotenv()

credentials = ServiceAccountCredentials.from_json_keyfile_name('gcreds.json')
client = storage.Client(credentials=credentials, project='assetgeneration')
bucket = client.get_bucket('xg_live_ops')

In [ ]:
blob = bucket.blob(f'p007/1/smaller_data.json')

In [ ]:
blob.public_url

In [ ]:
import json

In [ ]:
bucket.blob(f'{input["user_id"]}/{input["user_session_id"]}/out.json').upload_from_string(data=json.dumps(dict(out)), content_type="application/json")

In [ ]:
d = json.loads(out['field_info_list'][0].model_dump_json())

In [ ]:
import src.flat.flat.main

In [ ]:
src.flat.flat.main.main("temp/p007/1/", "original_copy.json")

In [ ]:
import json 

In [ ]:
def flatten_json_leaving_lists(data, parent_key='', sep='.'):
    """
    Flatten JSON data into a list of flat dictionaries, leaving list values as is.

    Parameters:
    data (dict, list, or any): The JSON data to flatten.
    parent_key (str): The base key string used during recursion.
    sep (str): Separator used when concatenating keys.

    Returns:
    list: A list of flat dictionaries.
    """
    flat_list = []

    if isinstance(data, dict):
        flat_dict = {}
        for k, v in data.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            if isinstance(v, dict):
                # Flatten nested dictionary
                nested_flat_list = flatten_json_leaving_lists(v, new_key, sep=sep)
                if nested_flat_list:
                    flat_dict.update(nested_flat_list[0])
            elif isinstance(v, list):
                # Leave list values as is
                flat_dict[new_key] = v
            else:
                # Primitive value
                flat_dict[new_key] = v
        flat_list.append(flat_dict)
    elif isinstance(data, list):
        # Process each item in the list
        for item in data:
            item_flat_list = flatten_json_leaving_lists(item, parent_key=parent_key, sep=sep)
            flat_list.extend(item_flat_list)
    else:
        # Data is a primitive value
        flat_list.append({parent_key or 'value': data})

    return flat_list


In [2]:
from itertools import islice
import json

In [3]:
def read_first_lines(file_path, num_lines):
    with open(file_path, 'r') as file:
        return list(islice(file, num_lines))

In [4]:
with open("temp/2b640d07-3ad2-472f-8c36-d2c4d62e9e92/6f4b4df5-08c8-4346-93a6-344a87bda899/out.json", "r") as f:
    data = json.loads(f.read())

In [5]:
data[0].keys()

dict_keys(['retained_player_events', 'player_event_history'])

In [ ]:
with open("src/flat/test_jsons/t2.json", "w") as f:
    json.dump(flatten_json_leaving_lists(json_data), f, indent=4)

In [ ]:
from pandas import DataFrame
import json

In [ ]:
user_id = "9fcd9778-dd90-435e-864c-d16d082273e9"
user_session_id = "9add04ac-9eb8-4c24-9eca-40ac833c76ef"

In [ ]:
def get_changes_to_field_names(USER_ID, USER_SESSION_ID) -> DataFrame:
    with open(f"""temp/{USER_ID}/{USER_SESSION_ID}/data_info.json""", "r") as f:
        data_info = json.load(f)

    changes = []
    for field_info in data_info:
        if field_info == "meaning_of_elements_in_data":
            continue
        changes.append({
            "field_name": data_info[field_info]['field_name'],
            "field_name_corrected": data_info[field_info]['field_new_name']
        })

    return DataFrame(changes)

In [ ]:
df = get_changes_to_field_names(user_id, user_session_id)

In [ ]:
df

In [ ]:
blob = bucket.blob(f'SJ5WRDaERqO0xmzrYsfgOi4qQH52/1dc4f238-8128-4cb6-a422-6749f053e334/field_name_changes.json')

In [2]:
import json


In [3]:
with open ("temp/323febd9-11b0-4d6b-b3c9-8b8e21898dd4/f339b2b7-b5d4-49fe-9731-60770f8b4510/data_info.json", "r") as f:
    data = json.load(f)

In [4]:
with open("temp/323febd9-11b0-4d6b-b3c9-8b8e21898dd4/f339b2b7-b5d4-49fe-9731-60770f8b4510/daily_retained_players/data_info.json", "r") as f:
    data1 = json.load(f)

In [5]:
with open("temp/323febd9-11b0-4d6b-b3c9-8b8e21898dd4/f339b2b7-b5d4-49fe-9731-60770f8b4510/d7_retained_player_activity_log/data_info.json", "r") as f:
    data2 = json.load(f)

In [6]:
with open ("temp/323febd9-11b0-4d6b-b3c9-8b8e21898dd4/f339b2b7-b5d4-49fe-9731-60770f8b4510/d7_retained_player_activity_log/player_event_log/data_info.json", "r") as f:
    data3 = json.load(f)

In [7]:
field_mapping = {}

In [19]:
for data_info in data3:
    print(data_info)
    if data_info == "meaning_of_elements_in_data":
        continue
    if data_info == 'parent_index_do_not_change':
        continue
    element = {}
    element['old_name'] = data3[data_info]['field_name']
    element['new_name'] = data3[data_info]['field_new_name']
    element['score'] = data3[data_info]['semantic_score']
    field_mapping[data3[data_info]['field_name']] = element

meaning_of_elements_in_data
parent_index_do_not_change
mp_country_code
event
victim_name
victim_type
distinct_id
trade-suggested
world
timestamp
time
server
advancement
username
entity_type
cause
trade-menu
reason
npc-reply
first-chat
item
item-take
item-give
block
message
entity


In [20]:
field_mapping

{'d7_retained': {'old_name': 'd7_retained',
  'new_name': 'd7_retained_player_activity_log',
  'score': 4.0},
 'd14_retained': {'old_name': 'd14_retained',
  'new_name': 'daily_retained_players',
  'score': 4.0},
 'events': {'old_name': 'events',
  'new_name': 'player_event_log',
  'score': 4.0},
 'player_id': {'old_name': 'player_id',
  'new_name': 'unique_player_identifier',
  'score': 4.0},
 'mp_country_code': {'old_name': 'mp_country_code',
  'new_name': 'player_country_code',
  'score': 4.0},
 'event': {'old_name': 'event', 'new_name': 'event_type', 'score': 4.0},
 'victim_name': {'old_name': 'victim_name',
  'new_name': 'chat_message_sender',
  'score': 4.0},
 'victim_type': {'old_name': 'victim_type',
  'new_name': 'victim_player_names',
  'score': 4.0},
 'distinct_id': {'old_name': 'distinct_id',
  'new_name': 'player_unique_identifier',
  'score': 4.0},
 'trade-suggested': {'old_name': 'trade-suggested',
  'new_name': 'suggested_trade_value',
  'score': 4.0},
 'world': {'old_n

In [21]:
with open("temp_data_info.json", "w") as f:
    json.dump(field_mapping, f, indent=4)    